# Projet info classes

In [ ]:
class Node:
    def __init__(self, lat : float, long : float):
        self.id = None # retrouver comment faire des identifiants uniques (avec un itérable?)
        self.lat = lat
        self.long = long
        self.children = []  # dico où les cles sont les identifiants, les valeurs [id,  et les valeurs le poids des aretes

    def new_child(self, node, vertex_value):  # crée un fils + donnée de sa distance au père
        ch = str(node.id)
        self.children[ch] = vertex_value
        ch2 = str(self.id)
        node.children[ch2] = vertex_value # a priori graphe non orienté (OS en premier lieu pas de sens interdits)

In [ ]:
class Client(Node): 
    def __init__(self, lat : float, long : float, colis : int):
        self.id = None # retrouver comment faire des identifiants uniques (avec un itérable?)
        self.lat = lat
        self.long = long
        self.children = []
        self.colis = colis # nombre de colis désirés

class Entrepot(Node):
    def __init__(self, lat : float, long : float, max_camions : int, max_legers : int, capacite : int):
        self.id = None # retrouver comment faire des identifiants uniques (avec un itérable?)
        self.lat = lat
        self.long = long
        self.children = []
        self.max_camions = max_camions
        self.max_legers = max_legers
        self.capacite = capacite

class Route: # j'aurais tendance à mettre ces infos (enfin etroite uniquement) dans le dico directement
    def __init__(self, vitesse, etroite : bool, dist):
        self.vitesse = vitesse
        self.etroite = etroite
        self.dist = dist

In [ ]:
class Vehicule:
    def __init__(self, capacite, pollution, dist_max, route_etroite : bool):
        self.capacite = capacite
        self.pollution = pollution
        self.dist_max = dist_max
        self.route_etroite = route_etroite
        
class Camion(Vehicule):
    def __init__(self, capacite, pollution, dist_max, route_etroite = False):
        self.capacite = capacite
        self.pollution = pollution
        self.dist_max = dist_max
        self.route_etroite = route_etroite
        
class Leger(Vehicule):
    def __init__(self, capacite, pollution = 0, dist_max = 10, route_etroite = True):
        self.capacite = capacite
        self.pollution = pollution
        self.dist_max = dist_max
        self.route_etroite = route_etroite    